In [ ]:
# Install the packages geemap, geopandas, earthengine-api, and geetools
pip install geemap geopandas earthengine-api geetools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 KB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 KB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 KB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 65.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━

In [ ]:
# Import the necessary packages 
import ee
from geetools import batch
import os
import geetools
from google.colab import drive
import geemap
import datetime
import glob
import matplotlib as mpl
import geopandas as gpd
from dateutil.relativedelta import relativedelta
from datetime import datetime
import numpy as npy 
from geemap import *

In [ ]:
# Mount your google drive
drive.mount('/content/drive/')
# Set the directory 
os.chdir('/content/drive/My Drive/Colab Notebooks/')

Mounted at /content/drive/


In [ ]:
# Authenticate Google Earth Engine. You need to have a Google Earth Engine account for this step. 
ee.Authenticate()

# Initialize your Goggle Earth Engine account
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize! Try to Authenticate EE')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=dVBEjG869dm2l61Gdl2YN1NhHj87-Hyf5wcGgRtt6Oc&tc=eejMsZeMtQGJdFLwTGP0-fGCqH-kZ5_AGfY9O_Cg-7I&cc=NnLgDYkgGrHEc_m52Q2RPkLT0KrmpL4Qs1lJAL4WERU

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7W4mk2GrNR9nDBOrFV5oGbt4CSSy-LsqEtGfJ4ZaXVojBx5Ft3jHY

Successfully saved authorization token.


In [25]:
# Collect GLDAS snow depth product and filter for the study period
SD = ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H').select('SnowDepth_inst').filterDate('2000-01-01', '2022-12-31')

# Collect Daymet SWE product and filter for the study period
SWE = ee.ImageCollection("NASA/ORNL/DAYMET_V4").select('swe').filterDate('2000-01-01', '2021-12-31')

In [26]:
# Set a path for importing the shapefiles that contain the study area
ANWR_path = ("/content/drive/My Drive/Colab Notebooks/Files/ANWR.shp")
NPRA_path = ("/content/drive/My Drive/Colab Notebooks/Files/NPRA.shp")

# Define the date range for each year
year_start = 2000
year_end = 2022

# Import shapefile for ANWR
# Import shapefile for NPRA
ANWR = geemap.shp_to_ee("/content/drive/My Drive/Colab Notebooks/Files/ANWR.shp")
NPRA = geemap.shp_to_ee("/content/drive/My Drive/Colab Notebooks/Files/NPRA.shp")

# Set the coordinate reference system (Projected coordinate system for United States (USA) - Alaska)
crs = 'EPSG:3338'

# Set a region (box) around ANWR
ANWR_region = ee.Geometry.Polygon(
    [[[-140.83089376368656,66.6550212415113],
      [-140.83089376368656,70.46793183571488],
      [-149.57601095118656,70.46793183571488],
      [-149.57601095118656,66.6550212415113]]], 
    None, False
)
# Set a region (box) around NPRA
NPRA_region = ee.Geometry.Polygon(
    [[[-149.90414749199374,67.92726660651483],
      [-149.90414749199374,71.81689549403126],
      [-163.43930374199374,71.81689549403126],
      [-163.43930374199374,67.92726660651483]]],
    None, False
)

In [27]:
# Define a function to create an image collection of the average (mean) monthly snow depth per pixel
def monthly_mean(imgC, path):
  # Convert the shapefile to an Earth Engine object
  studyArea = geemap.shp_to_ee(path)
  # Create an empty list that will be used to store the monthly means of the image collection 
  imageList = ee.List([])
  
  for m in range(1,13):
    # Filter the image collection by month
    data_month = imgC.filter(ee.Filter.calendarRange(m,m,'month'))
    # Calculate the mean per pixel
    data_reduce = data_month.mean()
    # Add the monthly mean to the empty list 
    imageList = imageList.add(data_reduce)
    # After the loop has finished, it creates an Earth Engine image collection from the images and assigns it to the variable "ImageCollection"
  ImageCollection = ee.ImageCollection.fromImages(imageList)
  # Clip each image to the study area
  ImageCollection_clip = ImageCollection.map(lambda img : img.clip(studyArea))
  # Return the clipped image collection
  return ImageCollection_clip

In [28]:
# Run the above function for both ANWR and NPRA for snow depth
SD_ANWR = monthly_mean(SD,ANWR_path)
SD_NPRA = monthly_mean(SD,NPRA_path)

# Run the above function for both ANWR and NPRA for SWE
SWE_ANWR = monthly_mean(SWE,ANWR_path)
SWE_NPRA = monthly_mean(SWE,NPRA_path)

In [31]:
# QA/QC step: Map the image collection to ensure that it clipped to the study area
# The max value for the SD = 1
# The max value for SWE = 100
Map=geemap.Map()

snowCoverVis = {'min': 0, 'max': 100, 'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']}
Map.addLayer(SD_ANWR,snowCoverVis,'snow_Depth_ANWR')
Map.addLayer(SD_NPRA,snowCoverVis,'snow_Depth_NPRA')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# **Create Snow Depth gif**

In [30]:
# Define arguments for animation function parameters.
ANWR_video_args = {
  'dimensions': 768,
  'region': ANWR_region,
  'framesPerSecond': 2,
  'crs': crs,
  'min': 0,
  'max': 1,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}

# Define arguments for animation function parameters.
NPRA_video_args = {
  'dimensions': 768,
  'region': NPRA_region,
  'framesPerSecond': 2,
  'crs': crs,
  'min': 0,
  'max': 1,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}

# Print the URL that will produce the animation for both NPRA and ANWR.
SD.ANWR_gif = SD_ANWR.getVideoThumbURL(ANWR_video_args)
print(SD.ANWR_gif)

SD.NPRA_gif = SD_NPRA.getVideoThumbURL(NPRA_video_args)
print(SD.NPRA_gif)

# Download ANWR animation to Google Drive.
import urllib.request
gif_name = 'SD_ANWR.gif'
urllib.request.urlretrieve(SD.ANWR_gif, gif_name)

# Download NPRA animation to Google Drive.
import urllib.request
gif_name = 'SD_NPRA.gif'
urllib.request.urlretrieve(SD.NPRA_gif, gif_name)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/273a6d54eddb94d4e72866880d30cd4f-870557217c71bc8ce7dcafacb2d4c9fd:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/ca543cd9cede725d8515914069a98ccb-924f700ded116b257c3b4f88bd817c08:getPixels


Add text and scale bar to the gif


*   Change the name of the imported/ exported gif to either NPRA or ANWR
*   Change text to either the NPRA or ANWR



In [ ]:
# Set path to import gif 
in_gif = os.path.abspath('../Colab Notebooks/SD_ANWR.gif')
# Set path to export gif
out_dir = os.path.join(os.path.expanduser('/content/drive/My Drive'), 'Colab Notebooks')
# Name the exported gif
out_gif = os.path.join(out_dir, 'ANWR_Depth_final.gif')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# Add text to gif
geemap.add_text_to_gif(
    in_gif, out_gif, xy=('5%', '3%'),font_size=40, text_sequence="Arctic National Wildlife Refuge", font_color='white',duration=500
)
geemap.add_text_to_gif(
    out_gif, out_gif, xy=('3%', '96%'),
    font_size=13, 
    text_sequence="*Average Snow Depth", 
    font_color='white',
    duration=500
)
# Create color bar
width = 250
height = 25
palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
labels = [0, 1]
colorbar = create_colorbar(
    width=width,
    height=height,
    palette=palette,
    vertical=True,
    add_labels=True,
    add_ticks=False,
    font_size=20,
    labels=labels,
    font_color='white'
)
# Add color bar to gif 
add_image_to_gif(
    out_gif, out_gif, in_image=colorbar, xy=('5%', '16%'), image_size=(250, 250)
)
show_image(out_gif)

# **Create SWE gif**

In [46]:
# Define arguments for animation function parameters.
ANWR_video_args = {
  'dimensions': 768,
  'region': ANWR_region,
  'framesPerSecond': 2,
  'crs': crs,
  'min': 0,
  'max': 100,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}

# Define arguments for animation function parameters.
NPRA_video_args = {
  'dimensions': 768,
  'region': NPRA_region,
  'framesPerSecond': 2,
  'crs': crs,
  'min': 0,
  'max': 100,
  'palette': ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
}

# Print the URL that will produce the animation for both NPRA and ANWR.
SWE.ANWR_gif = SWE_ANWR.getVideoThumbURL(ANWR_video_args)
print(SWE.ANWR_gif)

SWE.NPRA_gif = SWE_NPRA.getVideoThumbURL(NPRA_video_args)
print(SWE.NPRA_gif)

# Download ANWR animation to Google Drive.
import urllib.request
gif_name = 'SWE_ANWR.gif'
urllib.request.urlretrieve(SWE.ANWR_gif, gif_name)

# Download NPRA animation to Google Drive.
import urllib.request
gif_name = 'SWE_NPRA.gif'
urllib.request.urlretrieve(SWE.NPRA_gif, gif_name)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/7c4e692c739190ad818d96d503f416c3-f2f94e6fba04aa3d460bb5a237b86501:getPixels
https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/27ef5205de4a55f3424e4790b1eb81bf-2da4f4cb243f19769aac4ccdba71d7db:getPixels


('SWE_NPRA.gif', <http.client.HTTPMessage at 0x7f0ad7560dc0>)

Add text and scale bar to the gif


*   Change the name of the imported/ exported gif to either NPRA or ANWR
*   Change text to either the NPRA or ANWR

In [ ]:
# Set path to import gif 
in_gif = os.path.abspath('../Colab Notebooks/SWE_ANWR.gif')
# Set path to export gif
out_dir = os.path.join(os.path.expanduser('/content/drive/My Drive'), 'Colab Notebooks')
# Name the exported gif
out_gif = os.path.join(out_dir, 'ANWR_SWE_final.gif')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# Add text to gif
geemap.add_text_to_gif(
    in_gif, out_gif, xy=('5%', '3%'),font_size=40, text_sequence="Arctic National Wildlife Refuge", font_color='white',duration=500
)
geemap.add_text_to_gif(
    out_gif, out_gif, xy=('3%', '96%'),
    font_size=13, 
    text_sequence="*Average Snow Depth", 
    font_color='white',
    duration=500
)
# Create color bar
width = 250
height = 25
palette = ['0D0887', '5B02A3', '9A179B', 'CB4678', 'EB7852', 'FBB32F']
labels = [0, 1]
colorbar = create_colorbar(
    width=width,
    height=height,
    palette=palette,
    vertical=True,
    add_labels=True,
    add_ticks=False,
    font_size=20,
    labels=labels,
    font_color='white'
)
# Add color bar to gif 
add_image_to_gif(
    out_gif, out_gif, in_image=colorbar, xy=('5%', '16%'), image_size=(250, 250)
)
show_image(out_gif)